## Setup and Import

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15, 20)

In [ ]:
# export
import cv2
import time
import torch

In [ ]:
# export 
import os, sys
base = "/Users/ckh/Documents/Poladrone/nb"
# base = "D:/YoloV5_Hui/poladrone_nb"
sys.path.append(base + "/ref/pytorchYOLOv4")

In [ ]:
# export 
from exp.nb_TrainingRunnner import *
from exp.nb_LoggingModule import custom_init_logger
from config.config import Cfg

from ref.pytorchYOLOv4.tool.utils import nms_cpu, bbox_iou

## Evaluation

### Load Data

In [ ]:
# data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/train_npt.txt"
data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/npt_test.txt"
# data_list = "D:/YoloV5_Hui/Dataset/train_npt.txt"
val_dataset = Detection_dataset(data_list, Cfg, val=True, val_keep_size=True)


### Load Model

In [ ]:
# export 
def load_model(weight_path, n_classes=1):
    ## load model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Load: {weight_path} | Device: {device}')
    
    model = Yolov4(None, n_classes=n_classes, inference=True)
    model = torch.nn.DataParallel(model)
    model.load_state_dict(torch.load(weight_path, map_location=device))
    return model


In [ ]:
model_path = "./weight/Yolov4_epoch1707.pth"
model = load_model(model_path)

## Function

### Preprocessing

In [ ]:
# export 
def preprocess_image(img_raw, targetSize=416):
    img = cv2.resize(img_raw, (targetSize,targetSize))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img).div(255.0)
    img = img.unsqueeze(0)
    return img

### Inferencing

In [ ]:
# export 
def predict_img(model, img, size):
    img = preprocess_image(img, targetSize=size)
    out = model(img)
    return out

In [ ]:
# !mkdir inference

def inference_index(index, size, postfix="", show=False, save=False):
    img_raw, gt = val_dataset[index]
    out = predict_img(model, img_raw, size)
    if show:
        out_img = visualize_yolo(img_raw, out, gt=gt, targetSize=size, conf_thresh=0.01)
    if save:
        cv2.imwrite("./inference/%d%s.jpg"%(index, postfix), out_img)
    return out

### Post processing

In [ ]:
# export 
def post_processing(img, conf_thresh, nms_thresh, output):

    # anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
    # num_anchors = 9
    # anchor_masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # strides = [8, 16, 32]
    # anchor_step = len(anchors) // num_anchors

#     t1 = time.time()

    if type(output).__name__ != 'ndarray':
        output = output.cpu().detach().numpy()

    # [batch, num, 4]
    box_array = output[:, :, :4]

    # [batch, num, num_classes]
    confs = output[:, :, 4:]

    # [batch, num, num_classes] --> [batch, num]
    max_conf = np.max(confs, axis=2)
    max_id = np.argmax(confs, axis=2)

#     t2 = time.time()

    bboxes_batch = []
    for i in range(box_array.shape[0]):
       
        argwhere = max_conf[i] > conf_thresh
        l_box_array = box_array[i, argwhere, :]
        l_max_conf = max_conf[i, argwhere]
        l_max_id = max_id[i, argwhere]

        keep = nms_cpu(l_box_array, l_max_conf, nms_thresh)
        
        bboxes = []
        if (keep.size > 0):
            l_box_array = l_box_array[keep, :]
            l_max_conf = l_max_conf[keep]
            l_max_id = l_max_id[keep]

            for j in range(l_box_array.shape[0]):
                bboxes.append([l_box_array[j, 0], l_box_array[j, 1], l_box_array[j, 2], l_box_array[j, 3], l_max_conf[j], l_max_conf[j], l_max_id[j]])
        
        bboxes_batch.append(bboxes)

#     t3 = time.time()

#     print('-----------------------------------')
#     print('       max and argmax : %f' % (t2 - t1))
#     print('                  nms : %f' % (t3 - t2))
#     print('Post processing total : %f' % (t3 - t1))
#     print('-----------------------------------')
    
    return bboxes_batch


### Visualize

In [ ]:
# export
from matplotlib import pyplot as plt
import matplotlib

def visualize_yolo(img_raw, yolo_out, conf_thresh= 0.5, gt=None, targetSize=1000):
    out = yolo_out
    img = img_raw.copy()
    
    border = 2
    font_size = 1.2
    img = cv2.resize(img, (targetSize,targetSize))
        
    if not out.size == 0:
        out_box = out[out[:,:,4] > conf_thresh]
        bboxes = out_box[:,0:4] * targetSize
        conf = out_box[:,4]
        x,y,w,h = bboxes[:,0], bboxes[:,1], bboxes[:,2]/2, bboxes[:,3]/2

        x1, x2, y1, y2 = x-w, x+w, y-h, y+h

        for b in zip(x1,y1,x2,y2, conf):
            c = b[4]
            b = list(map(int,b))
            cv2.rectangle(img, (b[0],b[1]), (b[2],b[3]), (0,0,255), border)
            text = "NPT %.1f"%(c*100) ##self.id_names[b[4]]
            cv2.putText(img, text, (b[0],b[1]), cv2.FONT_HERSHEY_PLAIN, font_size, (255,0,0), border)
        
    if not gt is None:
        gt_box = gt[:,:4] * (targetSize / img_raw.shape[0])
        for g in gt_box:
            b = list(map(int,g))
            cv2.rectangle(img, (b[0],b[1]), (b[2],b[3]), (0,255,255), border)
            text = "GT" ##self.id_names[b[4]]
            cv2.putText(img, text, (b[2],b[1]), cv2.FONT_HERSHEY_PLAIN, font_size, (0,255,255), border)

    plt.imshow(img)
    
    return img

### Score

In [ ]:
# export 
def bbox_iogt(box1, box2, x1y1x2y2=True):
    '''
    box1: GT
    '''
    
    # print('iou box1:', box1)
    # print('iou box2:', box2)

    if x1y1x2y2:
        mx = min(box1[0], box2[0])
        Mx = max(box1[2], box2[2])
        my = min(box1[1], box2[1])
        My = max(box1[3], box2[3])
        w1 = box1[2] - box1[0]
        h1 = box1[3] - box1[1]
        w2 = box2[2] - box2[0]
        h2 = box2[3] - box2[1]
    else:
        w1 = box1[2]
        h1 = box1[3]
        w2 = box2[2]
        h2 = box2[3]

        mx = min(box1[0], box2[0])
        Mx = max(box1[0] + w1, box2[0] + w2)
        my = min(box1[1], box2[1])
        My = max(box1[1] + h1, box2[1] + h2)
    uw = Mx - mx
    uh = My - my
    cw = w1 + w2 - uw
    ch = h1 + h2 - uh
    carea = 0
    if cw <= 0 or ch <= 0:
        return 0.0

    area1 = w1 * h1
    area2 = w2 * h2
    carea = cw * ch
#     uarea = area1 + area2 - carea
    uarea = area1 
    return carea / uarea


In [ ]:
# export 
def score(gt, nms):
    if nms.size == 0:
        return 0, len(gt)
        
    box_gt = gt/1000

    x1 = nms[:,:,0] - nms[:,:,2]
    y1 = nms[:,:,1] - nms[:,:,3]
    x2 = nms[:,:,0] + nms[:,:,2]
    y2 = nms[:,:,1] + nms[:,:,3]
    box_predict = np.concatenate([x1,y1,x2,y2], axis=0).T

    iogt = [[bbox_iogt(g, p) for g in box_gt] for p in box_predict]
    iogt = np.max(iogt, axis=0)

    total = len(iogt)
    score = sum(iogt > 0.9)
    
    return score, total

In [ ]:
# export 
def recall(gt, nms):
    if nms.size == 0:
        if len(gt) == 0:
            return 1, 1
        return 0, 1
    
    box_gt = gt/1000
    x1 = nms[:,:,0] - nms[:,:,2]
    y1 = nms[:,:,1] - nms[:,:,3]
    x2 = nms[:,:,0] + nms[:,:,2]
    y2 = nms[:,:,1] + nms[:,:,3]
    box_predict = np.concatenate([x1,y1,x2,y2], axis=0).T
    
    iogt = [[bbox_iogt(g, p) for g in box_gt] for p in box_predict]
    iogt = np.max(iogt, axis=1)

    total = len(iogt)
    score = sum(iogt > 0.9)
    
    return score, total
    

## Setup Run all Above

## Predict - Experiment

#### Predict Flow

#### Comparing  input Size

In [ ]:
## Try with different size
index = 180
for s in range(0,300,16):
    size = 608+s
    inference_index(index,size=size, postfix="_%d"%(size))

In [ ]:
visualize_yolo(img_raw, out, conf_thresh=0.01, gt=gt, targetSize=416)

#### Comparing NMS

In [ ]:
## inference
index=180
out = predict_index(index, 720)  

In [ ]:
## Comparing NMS
nms_box = post_processing(None, 0.01,0.5, out)
nms = np.array(nms_box)
plt.subplot(121)
out_img = visualize_yolo(val_dataset[index][0], nms, 0, gt=val_dataset[index][1], targetSize=416)

nms_box = post_processing(None, 0.3,0.5, out)
nms = np.array(nms_box)
plt.subplot(122)
out_img = visualize_yolo(val_dataset[index][0], nms, 0, gt=val_dataset[index][1], targetSize=416)

#### Recall

In [ ]:
index = 10
size = 416
img_raw, gt = val_dataset[index]
out = predict_img(model, img_raw, size)

In [ ]:
nms_box = post_processing(None, 0.01,0.5, out)
nms = np.array(nms_box)
p, c = recall(gt, nms)
p, c

## Evaluation Flow - Main

In [ ]:
# export 
def benchmark_size(size):
    
    pt = []
    count = []
    t = []
    for index in range(len(val_dataset)):
        print(index, end=" ")
        ## inference
        img_raw, gt = val_dataset[index]

        t1 = time.time()
        out = predict_img(model, img_raw, size)
        t2 = time.time()
        t.append(t2 - t1)

        ## NMS
        nms_box = post_processing(None, 0.01,0.5, out)
        nms = np.array(nms_box)

        p, c = score(gt, nms)
        pt.append(p)
        count.append(c)

    inference_time = np.mean(t)
    score_pre_img = (np.array(pt) / np.array(count)).mean()
    score_all = np.sum(pt) / np.sum(count)

    return inference_time, score_pre_img, score_all

In [ ]:
data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/npt_test.txt"
# data_list = "D:/YoloV5_Hui/Dataset/train_npt.txt"
val_dataset = Detection_dataset(data_list, Cfg, val=True, val_keep_size=True)

In [ ]:
model_path = "./weight/Yolov4_epoch1707.pth"
model = load_model(model_path)

In [ ]:
logging = custom_init_logger("score")
logging_plot = custom_init_logger("plot")

for size in range(320,900,32):
    logging.info(f"Size : {size}")
    time_taken, score_img, score_all = benchmark_size(size)
    logging.info(f"""
                 Time : {time_taken} / image
                 Score per image: {score_img}
                 Score total : {score_all}
                 """)
    logging_plot.info(f"{size},{time_taken},{score_img},{score_all}")

## Graph Plot

### Size

In [ ]:
## Read txt
data_txt = "./result/Train1_size.txt"

def plot_size(txt):
    with open(txt, 'r') as f:
        lines = f.readlines()
        data = [l.rstrip().split(',') for l in lines]
        data = np.array(data, dtype=float)

    ## plot
    plt.subplot(121)
    plt.title("Time / Size")
    plt.plot(data[:,0], data[:,1], marker="o")

    plt.subplot(122)
    plt.title("Score / Size")
    plt.plot(data[:,0], data[:,2], marker="o")
    plt.plot(data[:,0], data[:,3], marker="o")
    
plot_size(data_txt)

In [ ]:
data_txt = "./result/Train2_size.txt"
plot_size(data_txt)

In [ ]:
data_txt = "./result/Train2_size_2.txt"
plot_size(data_txt)

### Threshold

In [ ]:
def visualize_thesh_data(txt, index=None):
    with open(data_txt, 'r') as f:
        lines = f.readlines()
        data = [l.rstrip().split(',') for l in lines]
        for d in data:
            d[2] = d[2].replace('[','').replace(']','').split(' ')
            d[3] = d[3].replace('[','').replace(']','').split(' ')

    size_data = np.array([d[0] for d in data], dtype=int)
    time_data = np.array([d[1] for d in data], dtype=float)
    score1_data = np.array([np.array(d[2]) for d in data], dtype=float)
    score2_data = np.array([d[3] for d in data], dtype=float)
    
    plot_thresh_score(size_data, score1_data, score2_data, index)
    
## plot
def plot_thresh_score(size_data, score1_data, score2_data, index=None):

    plt.subplot(221)
    plt.title("ALL : Score / Conf Threshold")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.plot(np.linspace(0.1,0.9,9), score1_data.T, marker="o")
    plt.legend(size_data)

    plt.title("ALL : Total Score / Conf Threshold")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.subplot(222)
    plt.plot(np.linspace(0.1,0.9,9), score2_data.T, marker="x")
    plt.legend(size_data)

    indexes = [3,12,13,14,15,16,17]
    if index:
        indexes = index

    plt.subplot(223)
    plt.title("Selected : Score / Conf Threshold")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
    plt.legend(size_data[indexes])

    plt.subplot(224)
    plt.title("Selected : Total Score / Conf Threshold")
    plt.xlabel("Threshold")
    plt.ylabel("Score")
    plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
    plt.legend(size_data[indexes])

    # indexes = [12,13,14,15]
    # plt.subplot(223)
    # plt.title("Selected : Score / Threshold")
    # plt.xlabel("Threshold")
    # plt.ylabel("Score")
    # plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
    # plt.legend(size_data[indexes])
    
    return

In [ ]:
data_txt = "./result/Train1_treshold.txt"
visualize_thesh_data(data_txt)

In [ ]:
data_txt = "./result/Train2_threshold.txt"
visualize_thesh_data(data_txt, index=[3,9,10,11,12])

### Plot Model_Size

In [ ]:
## model selection
data_txt = "./result/Train2_model_size_2.txt"

with open(data_txt, 'r') as f:
    lines = f.readlines()
    data = [l.rstrip().split(',') for l in lines]
    
model_data = {}
for d in data:
    if "Model" in d[0]:
        key = d[0].split(" : ")[1]
        model_data[key] = []
    else:
        d[2] = d[2].replace('[','').replace(']','').split(' ')
        d[3] = d[3].replace('[','').replace(']','').split(' ')
        model_data[key].append(d)
        
all_data = []
for k in list(model_data.keys()):
    for i in range(len(model_data[k])):
        model_data[k][i][0] = k + "-" + model_data[k][i][0]
        all_data.append(model_data[k][i])
                
data = all_data

size_data = np.array([d[0] for d in data], dtype=str)
time_data = np.array([d[1] for d in data], dtype=float)
score1_data = np.array([np.array(d[2]) for d in data], dtype=float)
score2_data = np.array([d[3] for d in data], dtype=float)
    
## plot

In [ ]:
indexes = np.where(['550-' in x for x in size_data])

plt.subplot(221)
plt.title("Selected : Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
plt.legend(size_data[indexes])

plt.subplot(222)
plt.title("Selected : Total Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
plt.legend(size_data[indexes])


In [ ]:
indexes = np.where(['-416' in x for x in size_data])

plt.subplot(221)
plt.title("Selected : Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
plt.legend(size_data[indexes])

plt.subplot(222)
plt.title("Selected : Total Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
plt.legend(size_data[indexes])

indexes = np.where(['-608' in x for x in size_data])
plt.subplot(223)
plt.title("Selected : Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
plt.legend(size_data[indexes])

plt.subplot(224)
plt.title("Selected : Total Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
plt.legend(size_data[indexes])

In [ ]:

indexes = np.where(['-640' in x for x in size_data])
plt.subplot(221)
plt.title("Selected : Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
plt.legend(size_data[indexes])

plt.subplot(222)
plt.title("Selected : Total Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
plt.legend(size_data[indexes])

indexes = np.where(['-672' in x for x in size_data])
plt.subplot(223)
plt.title("Selected : Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score1_data[indexes].T, marker="o")
plt.legend(size_data[indexes])

plt.subplot(224)
plt.title("Selected : Total Score / Conf Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.plot(np.linspace(0.1,0.9,9), score2_data[indexes].T, marker="x")
plt.legend(size_data[indexes])

### Breakdown

In [ ]:
def breakdownplot(size='416'):
    plt.figure()
    fig, ax = plt.subplots(2,3,sharey="row")
    query = f"-{size}"
    
    sep = 8
    indexes = np.where([query in x for x in size_data])[0][sep*0:sep*1]
    indexes1 = np.where([query in x for x in size_data])[0][sep*1:sep*2]
#     indexes2 = np.where([query in x for x in size_data])[0][sep*2:sep*3]

    for i, e in enumerate([indexes, indexes1]):
#         plt.subplot(2,3,1+i)
        plt.subplot(ax[0,i])
        plt.title(f"{size} Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score1_data[e].T
        plt.plot(x, y, marker="o")
        plt.legend(size_data[e])

    for i, e in enumerate([indexes, indexes1]):
#         plt.subplot(2,3,4+i)
        plt.subplot(ax[1,i])
        plt.title(f"{size} Total Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score2_data[e].T
        plt.plot(x, y, marker="o")
        plt.legend(size_data[e])

In [ ]:
breakdownplot(416)
breakdownplot(608)
breakdownplot(640)
breakdownplot(672)
breakdownplot(704)

### Selected Model

In [ ]:
def plot_model_breakdown(models=['1000']):
    plt.figure()
    fig, ax = plt.subplots(2,len(models),sharey="row")


    for i, model in enumerate(models):
        query = f"{model}-"
        indexes = np.where([query in x for x in size_data])

        plt.subplot(ax[0,i])
        plt.title(f"{model} Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score1_data[indexes].T
        plt.plot(x, y, marker="o")
        plt.legend(size_data[indexes])

        
        plt.subplot(ax[1,i])
        plt.title(f"{model} Total Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score2_data[indexes].T
        plt.plot(x, y, marker="o")
        plt.legend(size_data[indexes])

In [ ]:
# plot_model_breakdown(['1000', '1500', '1800', '2700', '3000'])
plot_model_breakdown(['300', '350', '550', '650', '800', '850'])

### plot Recall

In [ ]:
## model selection
data_txt = "./result/Train1_mode_size_recall.txt"

with open(data_txt, 'r') as f:
    lines = f.readlines()
    data = [l.rstrip().split(',') for l in lines]
    
model_data = {}
for d in data:
    if "Model" in d[0]:
        key = d[0].split(" : ")[1]
        model_data[key] = []
    else:
        d[2] = d[2].replace('[','').replace(']','').split(' ')
        d[3] = d[3].replace('[','').replace(']','').split(' ')
        d[4] = d[4].replace('[','').replace(']','').split(' ')
        d[5] = d[5].replace('[','').replace(']','').split(' ')
        model_data[key].append(d)
        
all_data = []
for k in list(model_data.keys()):
    for i in range(len(model_data[k])):
        model_data[k][i][0] = k + "-" + model_data[k][i][0]
        all_data.append(model_data[k][i])
                
data = all_data

size_data = np.array([d[0] for d in data], dtype=str)
time_data = np.array([d[1] for d in data], dtype=float)
score1_data = np.array([d[2] for d in data], dtype=float)
score2_data = np.array([d[3] for d in data], dtype=float)
recall1_data = np.array([d[4] for d in data], dtype=float)
recall2_data = np.array([d[5] for d in data], dtype=float)

    
## plot

In [ ]:
## selected Model with Recall Rate

# 1 - count ( result. max IOU with GT < threshole ) / total result
    

In [ ]:
def plot_model_recall(models=['1000']):
    plt.figure()
    fig, ax = plt.subplots(2,len(models),sharey="row")


    for i, model in enumerate(models):
        query = f"{model}-"
        indexes = np.where([query in x for x in size_data])

        plt.subplot(ax[0,i])
        plt.title(f"{model} Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score1_data[indexes].T
        l1 = plt.plot(x, y, marker="o")
        legend1 = plt.legend(l1, size_data[indexes], loc=1)
        plt.gca().add_artist(legend1)
        
        x = np.linspace(0.1,0.9,9)
        y = recall1_data[indexes].T
        l2 = plt.plot(x, y, marker="x")
        
        legend2 = plt.legend(l2, size_data[indexes], loc=4)
        plt.gca().add_artist(legend2)
        
        plt.subplot(ax[1,i])
        plt.title(f"{model} Total Score / Conf Threshold")
        plt.xlabel("Threshold")
        plt.ylabel("Score")
        x = np.linspace(0.1,0.9,9)
        y = score2_data[indexes].T
        l3 = plt.plot(x, y, marker="o")
        
        legend3 = plt.legend(l3, size_data[indexes], loc=1)
        plt.gca().add_artist(legend3)
        
        x = np.linspace(0.1,0.9,9)
        y = recall2_data[indexes].T
        l4 = plt.plot(x, y, marker="x")
        
        legend4 = plt.legend(l4, size_data[indexes], loc=4)
        plt.gca().add_artist(legend4)

In [ ]:
plot_model_recall(['1000', '1500', '1800', '2700', '3000'])

In [ ]:
data_txt = "./result/Train2_model_size_2.txt"

with open(data_txt, 'r') as f:
    lines = f.readlines()
    data = [l.rstrip().split(',') for l in lines]
    
model_data = {}
for d in data:
    if "Model" in d[0]:
        key = d[0].split(" : ")[1]
        model_data[key] = []
    else:
        d[2] = d[2].replace('[','').replace(']','').split(' ')
        d[3] = d[3].replace('[','').replace(']','').split(' ')
        d[4] = d[4].replace('[','').replace(']','').split(' ')
        d[5] = d[5].replace('[','').replace(']','').split(' ')
        model_data[key].append(d)
        
all_data = []
for k in list(model_data.keys()):
    for i in range(len(model_data[k])):
        model_data[k][i][0] = k + "-" + model_data[k][i][0]
        all_data.append(model_data[k][i])
                
data = all_data

size_data = np.array([d[0] for d in data], dtype=str)
time_data = np.array([d[1] for d in data], dtype=float)
score1_data = np.array([d[2] for d in data], dtype=float)
score2_data = np.array([d[3] for d in data], dtype=float)
recall1_data = np.array([d[4] for d in data], dtype=float)
recall2_data = np.array([d[5] for d in data], dtype=float)

    
## plot

In [ ]:
plot_model_recall(['300', '350', '550'])
plot_model_recall(['650', '800', '850'])

## Export

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NOTEBOOK = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
!python notebook2script.py "$NOTEBOOK".ipynb